<a href="https://colab.research.google.com/github/Jack3690/pydol/blob/main/notebooks/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install git+https://github.com/Jack3690/pydol/ -q
!pip install astroquery -q

  Preparing metadata (setup.py) ... done


In [21]:
from glob import glob
import astropy.io.fits as fits

import os
import astroquery
from astroquery.mast import Observations
import pandas as pd

from astropy.table import Table
from astropy.wcs import WCS
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import json
from astropy.coordinates import angular_separation
import astropy.units as u
from pydol.pipeline.jwst import jpipe
import multiprocessing as mp

# **Data**

**Observation ID format**

jw\<ppppp>\<ooo><vvv>\_\<gg>\<s>\<aa>\_\<eeeee>(-<"seg"NNN>)\_\<detector>\_\<prodType>.fits

where

ppppp: program ID number

ooo: observation number

vvv: visit number

gg: visit group

s: parallel sequence ID (1=prime, 2-5=parallel)

aa: activity number (base 36)

eeeee: exposure number

segNNN: the text “seg” followed by a three-digit segment number (optional)

detector: detector name (e.g. ‘nrca1’, ‘nrcblong’, ‘mirimage’)

prodType: product type identifier (e.g. ‘uncal’, ‘rate’, ‘cal’)

In [ ]:
#Example data for M82

mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '/content/data/stage0/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

ext = 'uncal'
# JWST images to be analyzed
image_files = []

# ngc 628: f"jw0178390400{k}_02101_0000{i}
for i in range(1,10):
    for j in range(1,5):
            for l in ['b']:
                image_files.append( f"jw01701052001_02102_0000{i}_nrc{l}{j}_uncal.fits")

for image_file in image_files:
    # Download file (if not already downloaded)
    mast_path  = os.path.join(mast_dir, image_file)
    local_path = os.path.join(data_dir, image_file)
    Observations.download_file(mast_path, local_path=local_path)

INFO:astroquery:Found cached file /content/data/stage0/jw01701052001_02102_00001_nrcb1_uncal.fits with expected size 4968000.
2024-06-26 21:51:48,883 - stpipe - INFO - Found cached file /content/data/stage0/jw01701052001_02102_00001_nrcb1_uncal.fits with expected size 4968000.
INFO:stpipe:Found cached file /content/data/stage0/jw01701052001_02102_00001_nrcb1_uncal.fits with expected size 4968000.


INFO: Found cached file /content/data/stage0/jw01701052001_02102_00001_nrcb1_uncal.fits with expected size 4968000. [astroquery.query]

# **Pipeline**

In [19]:
input_files = glob(f'/content/data/stage0/*')
out_dir = '/content/'
jwst_data = jpipe(input_files, out_dir,'jwst_1241.pmap')
jwst_data()

# **DOLPHOT Photometry**